# 🐍🗄️ MÓDULO 3.2: PYTHON + SQLITE
## 📊 Maestría en Python - Fase 3: Integración de Datos

---

### 🎯 OBJETIVOS DE APRENDIZAJE

Al completar este módulo serás capaz de:
- ✅ Integrar Python con SQLite de forma profesional
- ✅ Implementar patrones DAO y Repository
- ✅ Crear sistemas de monitoreo industrial en tiempo real
- ✅ Automatizar reportes y análisis de datos
- ✅ Usar Pandas para análisis híbrido SQL-Python
- ✅ Implementar seguridad y mejores prácticas

### 📋 TEMARIO DETALLADO

1. **Conexión Python-SQLite** (sqlite3, context managers)
2. **Operaciones CRUD** (Create, Read, Update, Delete)
3. **Patrones de Diseño** (DAO, Repository, Factory)
4. **Pandas + SQL** (Análisis híbrido de datos)
5. **Sistema de Monitoreo Industrial** (Tiempo real, alertas)
6. **Automatización de Reportes** (Scheduling, exports)
7. **Optimización y Seguridad** (Pool conexiones, validación)
8. **Proyecto Integrador** (Dashboard industrial completo)

---

## 🔌 1. CONEXIÓN PYTHON-SQLITE

### 🤝 Integrando los Mundos SQL y Python

Python incluye **sqlite3** de forma nativa, permitiendo una integración perfecta con bases de datos. Aprenderemos desde conexiones básicas hasta patrones profesionales.

### 🏗️ Fundamentos de Conexión

In [ ]:
# 🚀 CONFIGURACIÓN INICIAL - Conexión Básica Python-SQLite
import sqlite3
import pandas as pd
from datetime import datetime, date
from contextlib import contextmanager
from typing import List, Dict, Any, Optional
import json
import random
from pathlib import Path

print("✅ Librerías importadas exitosamente")
print(f"📅 Sesión iniciada: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Versión Python con sqlite3 integrado")

In [ ]:
# 🔗 CONEXIÓN BÁSICA A SQLITE
def conexion_basica_demo():
    """Demostración de conexión básica"""
    
    # Crear/conectar a base de datos
    conn = sqlite3.connect('empresa_industrial.db')
    print("✅ Conexión establecida a 'empresa_industrial.db'")
    
    # Configurar para resultados como diccionarios
    conn.row_factory = sqlite3.Row
    print("🔧 Configurado row_factory para resultados tipo diccionario")
    
    # Crear cursor para ejecutar comandos
    cursor = conn.cursor()
    
    # Verificar versión de SQLite
    cursor.execute("SELECT sqlite_version()")
    version = cursor.fetchone()[0]
    print(f"📊 Versión SQLite: {version}")
    
    # Listar tablas existentes
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tablas = cursor.fetchall()
    print(f"📋 Tablas existentes: {[tabla[0] for tabla in tablas]}")
    
    # ¡IMPORTANTE! Cerrar conexión
    conn.close()
    print("🔒 Conexión cerrada correctamente")
    
    return version

# Ejecutar demostración
version_sqlite = conexion_basica_demo()

print("\n" + "="*50)
print("🧠 CONCEPTOS CLAVE:")
print("✅ sqlite3.connect() - Establece conexión")
print("✅ row_factory - Configura formato de resultados")
print("✅ cursor() - Ejecuta comandos SQL")
print("✅ conn.close() - Libera recursos")
print("="*50)

### 🛡️ Context Managers - Gestión Segura de Conexiones

Los **context managers** garantizan que las conexiones se cierren automáticamente, incluso si ocurre un error. Esto es **crítico** en aplicaciones industriales.

In [ ]:
# 🛡️ CONTEXT MANAGER PROFESIONAL
@contextmanager
def obtener_conexion_segura(db_path: str):
    """Context manager para manejo seguro de conexiones"""
    conn = None
    try:
        print(f"🔗 Estableciendo conexión a {db_path}")
        conn = sqlite3.connect(db_path)
        conn.row_factory = sqlite3.Row  # Resultados como diccionarios
        print("✅ Conexión establecida y configurada")
        yield conn
    except sqlite3.Error as e:
        print(f"❌ Error en base de datos: {e}")
        if conn:
            conn.rollback()
            print("🔄 Rollback ejecutado")
        raise
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        if conn:
            conn.rollback()
        raise
    finally:
        if conn:
            conn.close()
            print("🔒 Conexión cerrada automáticamente")

# Demostración del context manager
def demo_context_manager():
    """Demuestra el uso del context manager"""
    
    print("🚀 DEMOSTRACIÓN CONTEXT MANAGER")
    print("-" * 40)
    
    # Uso normal (exitoso)
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT 'Operación exitosa' as mensaje")
            resultado = cursor.fetchone()
            print(f"📋 Resultado: {resultado['mensaje']}")
            # La conexión se cierra automáticamente aquí
    except Exception as e:
        print(f"Error capturado: {e}")
    
    print("\n🧪 Simulando error para mostrar rollback automático:")
    
    # Simulación de error
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            # SQL inválido para forzar error
            cursor.execute("SELECT FROM tabla_inexistente")
    except Exception as e:
        print(f"🎯 Error capturado y manejado: {type(e).__name__}")
    
    print("\n✅ Demostración completada - Conexiones manejadas seguramente")

# Ejecutar demostración
demo_context_manager()

print("\n" + "="*50)
print("🧠 VENTAJAS DEL CONTEXT MANAGER:")
print("✅ Cierre automático de conexiones")
print("✅ Rollback automático en errores")
print("✅ Manejo consistente de excepciones")
print("✅ Código más limpio y profesional")
print("="*50)

## 📊 2. OPERACIONES CRUD CON PYTHON

### 🔧 Create, Read, Update, Delete

Implementaremos las operaciones fundamentales CRUD usando Python, integrando los conocimientos SQL del módulo anterior con patrones de programación profesionales.

In [ ]:
# 🏗️ CONFIGURACIÓN DE TABLAS PARA CRUD
def inicializar_sistema_empleados():
    """Inicializa el sistema de gestión de empleados"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        
        # Crear tabla empleados con todas las validaciones
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS empleados (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nombre TEXT NOT NULL CHECK(length(nombre) > 0),
                apellido TEXT NOT NULL CHECK(length(apellido) > 0),
                email TEXT UNIQUE NOT NULL CHECK(email LIKE '%@%'),
                telefono TEXT CHECK(length(telefono) >= 10),
                salario REAL CHECK(salario > 0),
                departamento TEXT NOT NULL,
                fecha_ingreso DATE NOT NULL,
                años_experiencia INTEGER CHECK(años_experiencia >= 0),
                activo BOOLEAN DEFAULT 1,
                fecha_creacion DATETIME DEFAULT CURRENT_TIMESTAMP,
                fecha_actualizacion DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Crear tabla de auditoría
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS auditoria_empleados (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                empleado_id INTEGER,
                operacion TEXT NOT NULL,
                datos_anteriores TEXT,
                datos_nuevos TEXT,
                usuario TEXT DEFAULT 'sistema',
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Crear índices para optimización
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_empleados_email ON empleados(email)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_empleados_departamento ON empleados(departamento)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_empleados_activo ON empleados(activo)')
        
        conn.commit()
        print("✅ Sistema de empleados inicializado")
        print("📋 Tablas creadas: empleados, auditoria_empleados")
        print("🚀 Índices optimizados creados")

# Inicializar sistema
inicializar_sistema_empleados()

# Verificar estructura de la tabla
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(empleados)")
    columnas = cursor.fetchall()
    
    print("\n📊 ESTRUCTURA DE LA TABLA EMPLEADOS:")
    print("-" * 60)
    for col in columnas:
        print(f"📄 {col['name']:<20} | {col['type']:<10} | {col['notnull']} | {col['dflt_value']}")
    
    print(f"\n🎯 Total de columnas: {len(columnas)}")

### ➕ CREATE - Inserción de Datos

Implementaremos inserción de datos con validación, manejo de errores y auditoría.

### ✏️ UPDATE - Actualización de Datos

Implementaremos actualizaciones con auditoría completa y validación de cambios.

In [ ]:
# ➕ OPERACIONES CREATE (INSERCIÓN)
def crear_empleado(datos_empleado: Dict[str, Any]) -> Optional[int]:
    """Crea un nuevo empleado con validación completa"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Validar email único
            cursor.execute("SELECT COUNT(*) FROM empleados WHERE email = ?", (datos_empleado['email'],))
            if cursor.fetchone()[0] > 0:
                print(f"❌ Error: Email {datos_empleado['email']} ya existe")
                return None
            
            # Insertar empleado
            cursor.execute('''
                INSERT INTO empleados (nombre, apellido, email, telefono, salario, 
                                     departamento, fecha_ingreso, años_experiencia)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                datos_empleado['nombre'],
                datos_empleado['apellido'],
                datos_empleado['email'],
                datos_empleado.get('telefono', ''),
                datos_empleado['salario'],
                datos_empleado['departamento'],
                datos_empleado['fecha_ingreso'],
                datos_empleado.get('años_experiencia', 0)
            ))
            
            empleado_id = cursor.lastrowid
            
            # Registrar auditoría
            cursor.execute('''
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                VALUES (?, 'CREATE', ?)
            ''', (empleado_id, json.dumps(datos_empleado)))
            
            conn.commit()
            print(f"✅ Empleado creado exitosamente con ID: {empleado_id}")
            return empleado_id
            
    except sqlite3.IntegrityError as e:
        print(f"❌ Error de integridad: {e}")
        return None
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return None

def crear_empleados_masivo(lista_empleados: List[Dict[str, Any]]) -> List[int]:
    """Crea múltiples empleados en una transacción"""
    
    empleados_creados = []
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            for empleado in lista_empleados:
                try:
                    cursor.execute('''
                        INSERT INTO empleados (nombre, apellido, email, telefono, salario, 
                                             departamento, fecha_ingreso, años_experiencia)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        empleado['nombre'], empleado['apellido'], empleado['email'],
                        empleado.get('telefono', ''), empleado['salario'],
                        empleado['departamento'], empleado['fecha_ingreso'],
                        empleado.get('años_experiencia', 0)
                    ))
                    
                    empleado_id = cursor.lastrowid
                    empleados_creados.append(empleado_id)
                    
                    # Auditoría
                    cursor.execute('''
                        INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                        VALUES (?, 'CREATE_BULK', ?)
                    ''', (empleado_id, json.dumps(empleado)))
                    
                except sqlite3.IntegrityError as e:
                    print(f"⚠️ Error al crear {empleado['email']}: {e}")
                    continue
            
            conn.commit()
            print(f"✅ Creación masiva completada: {len(empleados_creados)} empleados")
            return empleados_creados
            
    except Exception as e:
        print(f"❌ Error en creación masiva: {e}")
        return []

# 🧪 PRUEBAS DE INSERCIÓN
print("🧪 PROBANDO OPERACIONES CREATE")
print("=" * 50)

# Empleado individual
empleado_prueba = {
    'nombre': 'Carlos',
    'apellido': 'Rodríguez',
    'email': 'carlos.rodriguez@industrial.com',
    'telefono': '555-0001',
    'salario': 75000.00,
    'departamento': 'Automatización',
    'fecha_ingreso': '2024-01-15',
    'años_experiencia': 8
}

id_carlos = crear_empleado(empleado_prueba)

# Empleados masivos
empleados_masivos = [
    {
        'nombre': 'María', 'apellido': 'González', 
        'email': 'maria.gonzalez@industrial.com',
        'salario': 68000.00, 'departamento': 'Ingeniería',
        'fecha_ingreso': '2023-03-20', 'años_experiencia': 5
    },
    {
        'nombre': 'Juan', 'apellido': 'Pérez',
        'email': 'juan.perez@industrial.com', 
        'salario': 82000.00, 'departamento': 'Automatización',
        'fecha_ingreso': '2022-07-10', 'años_experiencia': 12
    },
    {
        'nombre': 'Ana', 'apellido': 'López',
        'email': 'ana.lopez@industrial.com',
        'salario': 59000.00, 'departamento': 'Calidad',
        'fecha_ingreso': '2024-02-01', 'años_experiencia': 3
    }
]

ids_creados = crear_empleados_masivo(empleados_masivos)

print(f"\n📊 RESUMEN DE CREACIÓN:")
print(f"👤 Empleado individual: ID {id_carlos}")
print(f"👥 Empleados masivos: {len(ids_creados)} creados")
print(f"🎯 IDs generados: {ids_creados}")

# Verificar total de empleados
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
    total = cursor.fetchone()[0]
    print(f"\n📈 Total empleados activos en sistema: {total}")

### 🔍 READ - Consulta de Datos

Implementaremos diferentes patrones de consulta, desde búsquedas simples hasta análisis complejos.

In [ ]:
# 🔍 OPERACIONES READ (CONSULTA)
def obtener_empleado_por_id(empleado_id: int) -> Optional[Dict[str, Any]]:
    """Obtiene un empleado específico por ID"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT * FROM empleados 
            WHERE id = ? AND activo = 1
        """, (empleado_id,))
        
        row = cursor.fetchone()
        return dict(row) if row else None

def buscar_empleados(filtros: Dict[str, Any] = None, orden: str = 'apellido') -> List[Dict[str, Any]]:
    """Búsqueda avanzada de empleados con filtros dinámicos"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        
        # Construir query dinámico
        where_clauses = ["activo = 1"]
        valores = []
        
        if filtros:
            for campo, valor in filtros.items():
                if isinstance(valor, str) and '%' in valor:
                    where_clauses.append(f"{campo} LIKE ?")
                elif isinstance(valor, dict) and 'min' in valor and 'max' in valor:
                    where_clauses.append(f"{campo} BETWEEN ? AND ?")
                    valores.extend([valor['min'], valor['max']])
                    continue
                elif isinstance(valor, list):
                    where_clauses.append(f"{campo} IN ({','.join(['?'] * len(valor))})")
                    valores.extend(valor)
                    continue
                else:
                    where_clauses.append(f"{campo} = ?")
                valores.append(valor)
        
        where_sql = " AND ".join(where_clauses)
        query = f"""
            SELECT * FROM empleados 
            WHERE {where_sql}
            ORDER BY {orden}
        """
        
        cursor.execute(query, valores)
        return [dict(row) for row in cursor.fetchall()]

def obtener_estadisticas_empleados() -> Dict[str, Any]:
    """Obtiene estadísticas completas del sistema"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        
        # Estadísticas generales
        cursor.execute("""
            SELECT 
                COUNT(*) as total_empleados,
                ROUND(AVG(salario), 2) as salario_promedio,
                MIN(salario) as salario_minimo,
                MAX(salario) as salario_maximo,
                SUM(salario) as nomina_total,
                ROUND(AVG(años_experiencia), 1) as experiencia_promedio
            FROM empleados WHERE activo = 1
        """)
        
        stats_generales = dict(cursor.fetchone())
        
        # Estadísticas por departamento
        cursor.execute("""
            SELECT 
                departamento,
                COUNT(*) as empleados,
                ROUND(AVG(salario), 2) as salario_promedio,
                SUM(salario) as costo_departamento
            FROM empleados 
            WHERE activo = 1
            GROUP BY departamento
            ORDER BY empleados DESC
        """)
        
        stats_departamento = [dict(row) for row in cursor.fetchall()]
        
        # Distribución por experiencia
        cursor.execute("""
            SELECT 
                CASE 
                    WHEN años_experiencia <= 2 THEN 'Junior (0-2 años)'
                    WHEN años_experiencia <= 5 THEN 'Semi-Senior (3-5 años)'
                    WHEN años_experiencia <= 10 THEN 'Senior (6-10 años)'
                    ELSE 'Experto (11+ años)'
                END as nivel,
                COUNT(*) as cantidad,
                ROUND(AVG(salario), 2) as salario_promedio
            FROM empleados 
            WHERE activo = 1
            GROUP BY nivel
            ORDER BY cantidad DESC
        """)
        
        distribucion_experiencia = [dict(row) for row in cursor.fetchall()]
        
        return {
            'generales': stats_generales,
            'por_departamento': stats_departamento,
            'por_experiencia': distribucion_experiencia,
            'timestamp': datetime.now().isoformat()
        }

# 🧪 PRUEBAS DE CONSULTA
print("🧪 PROBANDO OPERACIONES READ")
print("=" * 50)

# Consulta por ID
if id_carlos:
    empleado = obtener_empleado_por_id(id_carlos)
    if empleado:
        print(f"👤 EMPLEADO ID {id_carlos}:")
        print(f"   Nombre: {empleado['nombre']} {empleado['apellido']}")
        print(f"   Email: {empleado['email']}")
        print(f"   Departamento: {empleado['departamento']}")
        print(f"   Salario: ${empleado['salario']:,.2f}")

# Búsqueda por departamento
print("\n🔍 EMPLEADOS DE AUTOMATIZACIÓN:")
empleados_auto = buscar_empleados({'departamento': 'Automatización'})
for emp in empleados_auto:
    print(f"   • {emp['nombre']} {emp['apellido']} - ${emp['salario']:,.2f}")

# Búsqueda por rango salarial
print("\n💰 EMPLEADOS CON SALARIO $60K - $80K:")
empleados_rango = buscar_empleados({'salario': {'min': 60000, 'max': 80000}})
for emp in empleados_rango:
    print(f"   • {emp['nombre']} {emp['apellido']} ({emp['departamento']}) - ${emp['salario']:,.2f}")

# Búsqueda con LIKE
print("\n🔎 EMPLEADOS CON NOMBRES QUE CONTIENEN 'ar':")
empleados_like = buscar_empleados({'nombre': '%ar%'})
for emp in empleados_like:
    print(f"   • {emp['nombre']} {emp['apellido']}")

# Estadísticas completas
print("\n📊 ESTADÍSTICAS DEL SISTEMA:")
stats = obtener_estadisticas_empleados()

print("\n📈 GENERALES:")
gen = stats['generales']
print(f"   👥 Total empleados: {gen['total_empleados']}")
print(f"   💰 Salario promedio: ${gen['salario_promedio']:,.2f}")
print(f"   📊 Rango salarial: ${gen['salario_minimo']:,.2f} - ${gen['salario_maximo']:,.2f}")
print(f"   💸 Nómina total: ${gen['nomina_total']:,.2f}")
print(f"   ⏱️ Experiencia promedio: {gen['experiencia_promedio']} años")

print("\n🏢 POR DEPARTAMENTO:")
for dept in stats['por_departamento']:
    print(f"   • {dept['departamento']}: {dept['empleados']} empleados, promedio ${dept['salario_promedio']:,.2f}")

print("\n🎯 POR NIVEL DE EXPERIENCIA:")
for nivel in stats['por_experiencia']:
    print(f"   • {nivel['nivel']}: {nivel['cantidad']} empleados, promedio ${nivel['salario_promedio']:,.2f}")

In [ ]:
# ✏️ OPERACIONES UPDATE (ACTUALIZACIÓN)
def actualizar_empleado(empleado_id: int, datos_nuevos: Dict[str, Any]) -> bool:
    """Actualiza un empleado con auditoría completa"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Obtener datos actuales para auditoría
            cursor.execute("SELECT * FROM empleados WHERE id = ? AND activo = 1", (empleado_id,))
            empleado_actual = cursor.fetchone()
            
            if not empleado_actual:
                print(f"❌ Empleado ID {empleado_id} no encontrado")
                return False
            
            datos_anteriores = dict(empleado_actual)
            
            # Construir query dinámico para actualización
            campos_actualizacion = []
            valores = []
            
            for campo, valor in datos_nuevos.items():
                if campo in ['id', 'fecha_creacion']:  # Campos protegidos
                    continue
                campos_actualizacion.append(f"{campo} = ?")
                valores.append(valor)
            
            if not campos_actualizacion:
                print("⚠️ No hay campos válidos para actualizar")
                return False
            
            # Agregar fecha de actualización
            campos_actualizacion.append("fecha_actualizacion = CURRENT_TIMESTAMP")
            valores.append(empleado_id)
            
            query = f"""
                UPDATE empleados 
                SET {', '.join(campos_actualizacion)}
                WHERE id = ? AND activo = 1
            """
            
            cursor.execute(query, valores)
            
            if cursor.rowcount == 0:
                print(f"❌ No se pudo actualizar empleado ID {empleado_id}")
                return False
            
            # Registrar auditoría
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_anteriores, datos_nuevos)
                VALUES (?, 'UPDATE', ?, ?)
            """, (empleado_id, json.dumps(datos_anteriores), json.dumps(datos_nuevos)))
            
            conn.commit()
            print(f"✅ Empleado ID {empleado_id} actualizado exitosamente")
            print(f"📝 Campos modificados: {list(datos_nuevos.keys())}")
            return True
            
    except sqlite3.IntegrityError as e:
        print(f"❌ Error de integridad al actualizar: {e}")
        return False
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return False

def actualizar_salarios_departamento(departamento: str, porcentaje_aumento: float) -> Dict[str, Any]:
    """Actualiza salarios de todo un departamento"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Obtener empleados del departamento
            cursor.execute("""
                SELECT id, nombre, apellido, salario 
                FROM empleados 
                WHERE departamento = ? AND activo = 1
            """, (departamento,))
            
            empleados = cursor.fetchall()
            
            if not empleados:
                print(f"❌ No se encontraron empleados en departamento: {departamento}")
                return {'success': False, 'empleados_actualizados': 0}
            
            empleados_actualizados = 0
            total_aumento = 0
            
            for empleado in empleados:
                emp_id, nombre, apellido, salario_actual = empleado
                nuevo_salario = salario_actual * (1 + porcentaje_aumento / 100)
                aumento = nuevo_salario - salario_actual
                
                # Actualizar salario individual
                cursor.execute("""
                    UPDATE empleados 
                    SET salario = ?, fecha_actualizacion = CURRENT_TIMESTAMP
                    WHERE id = ?
                """, (nuevo_salario, emp_id))
                
                # Auditoría
                cursor.execute("""
                    INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                    VALUES (?, 'SALARY_UPDATE', ?)
                """, (emp_id, json.dumps({
                    'salario_anterior': salario_actual,
                    'salario_nuevo': nuevo_salario,
                    'aumento_porcentaje': porcentaje_aumento,
                    'aumento_valor': aumento,
                    'departamento': departamento
                })))
                
                empleados_actualizados += 1
                total_aumento += aumento
                
                print(f"💰 {nombre} {apellido}: ${salario_actual:,.2f} → ${nuevo_salario:,.2f} (+${aumento:,.2f})")
            
            conn.commit()
            
            resultado = {
                'success': True,
                'departamento': departamento,
                'empleados_actualizados': empleados_actualizados,
                'porcentaje_aumento': porcentaje_aumento,
                'total_aumento': total_aumento,
                'promedio_aumento': total_aumento / empleados_actualizados if empleados_actualizados > 0 else 0
            }
            
            print(f"✅ Actualización masiva completada:")
            print(f"   📊 Departamento: {departamento}")
            print(f"   👥 Empleados actualizados: {empleados_actualizados}")
            print(f"   📈 Aumento total: ${total_aumento:,.2f}")
            print(f"   📊 Aumento promedio: ${resultado['promedio_aumento']:,.2f}")
            
            return resultado
            
    except Exception as e:
        print(f"❌ Error en actualización masiva: {e}")
        return {'success': False, 'error': str(e)}

# 🧪 PRUEBAS DE ACTUALIZACIÓN
print("🧪 PROBANDO OPERACIONES UPDATE")
print("=" * 50)

# Actualización individual
if id_carlos:
    print(f"📝 ACTUALIZANDO EMPLEADO ID {id_carlos}:")
    
    # Mostrar datos actuales
    empleado_actual = obtener_empleado_por_id(id_carlos)
    if empleado_actual:
        print(f"   📋 Datos actuales:")
        print(f"      Teléfono: {empleado_actual['telefono']}")
        print(f"      Salario: ${empleado_actual['salario']:,.2f}")
        print(f"      Experiencia: {empleado_actual['años_experiencia']} años")
    
    # Actualizar datos
    nuevos_datos = {
        'telefono': '555-9999',
        'salario': 78000.00,
        'años_experiencia': 9
    }
    
    success = actualizar_empleado(id_carlos, nuevos_datos)
    
    if success:
        # Mostrar datos actualizados
        empleado_actualizado = obtener_empleado_por_id(id_carlos)
        print(f"   📋 Datos actualizados:")
        print(f"      Teléfono: {empleado_actualizado['telefono']}")
        print(f"      Salario: ${empleado_actualizado['salario']:,.2f}")
        print(f"      Experiencia: {empleado_actualizado['años_experiencia']} años")

# Actualización masiva por departamento
print(f"\n💼 APLICANDO AUMENTO DEL 5% AL DEPARTAMENTO DE AUTOMATIZACIÓN:")
resultado_aumento = actualizar_salarios_departamento('Automatización', 5.0)

# Verificar cambios
print(f"\n📊 VERIFICACIÓN POST-ACTUALIZACIÓN:")
empleados_auto_actualizados = buscar_empleados({'departamento': 'Automatización'})
for emp in empleados_auto_actualizados:
    print(f"   • {emp['nombre']} {emp['apellido']}: ${emp['salario']:,.2f}")

# Consultar auditoría
print(f"\n📋 ÚLTIMAS OPERACIONES DE AUDITORÍA:")
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT a.*, e.nombre, e.apellido 
        FROM auditoria_empleados a
        LEFT JOIN empleados e ON a.empleado_id = e.id
        ORDER BY a.timestamp DESC 
        LIMIT 5
    """)
    
    auditoria = cursor.fetchall()
    for registro in auditoria:
        timestamp = registro['timestamp']
        operacion = registro['operacion']
        empleado = f"{registro['nombre']} {registro['apellido']}" if registro['nombre'] else "N/A"
        print(f"   📝 {timestamp}: {operacion} - {empleado}")

print("\n🧠 CONCEPTOS CLAVE UPDATE:")
print("✅ Auditoría completa de cambios")
print("✅ Validación de campos protegidos")  
print("✅ Actualización masiva con transacciones")
print("✅ Queries dinámicos para flexibilidad")
print("✅ Manejo robusto de errores")

### 🗑️ DELETE - Eliminación Segura

Implementaremos eliminación lógica (soft delete) y física, siempre con auditoría.

In [ ]:
# 🗑️ OPERACIONES DELETE (ELIMINACIÓN)
def eliminar_empleado_logico(empleado_id: int, motivo: str = "No especificado") -> bool:
    """Eliminación lógica (soft delete) - Marca como inactivo"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Verificar que el empleado existe y está activo
            cursor.execute("SELECT * FROM empleados WHERE id = ? AND activo = 1", (empleado_id,))
            empleado = cursor.fetchone()
            
            if not empleado:
                print(f"❌ Empleado ID {empleado_id} no encontrado o ya inactivo")
                return False
            
            datos_empleado = dict(empleado)
            
            # Marcar como inactivo
            cursor.execute("""
                UPDATE empleados 
                SET activo = 0, fecha_actualizacion = CURRENT_TIMESTAMP
                WHERE id = ?
            """, (empleado_id,))
            
            # Registrar auditoría
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_anteriores, datos_nuevos)
                VALUES (?, 'SOFT_DELETE', ?, ?)
            """, (empleado_id, json.dumps(datos_empleado), json.dumps({'motivo': motivo})))
            
            conn.commit()
            print(f"✅ Empleado ID {empleado_id} marcado como inactivo")
            print(f"📝 Motivo: {motivo}")
            print(f"👤 Empleado: {datos_empleado['nombre']} {datos_empleado['apellido']}")
            return True
            
    except Exception as e:
        print(f"❌ Error en eliminación lógica: {e}")
        return False

def restaurar_empleado(empleado_id: int) -> bool:
    """Restaura un empleado marcado como inactivo"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Verificar que el empleado existe y está inactivo
            cursor.execute("SELECT * FROM empleados WHERE id = ? AND activo = 0", (empleado_id,))
            empleado = cursor.fetchone()
            
            if not empleado:
                print(f"❌ Empleado ID {empleado_id} no encontrado o ya activo")
                return False
            
            datos_empleado = dict(empleado)
            
            # Reactivar empleado
            cursor.execute("""
                UPDATE empleados 
                SET activo = 1, fecha_actualizacion = CURRENT_TIMESTAMP
                WHERE id = ?
            """, (empleado_id,))
            
            # Registrar auditoría
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                VALUES (?, 'RESTORE', ?)
            """, (empleado_id, json.dumps({'restaurado': True})))
            
            conn.commit()
            print(f"✅ Empleado ID {empleado_id} restaurado exitosamente")
            print(f"👤 Empleado: {datos_empleado['nombre']} {datos_empleado['apellido']}")
            return True
            
    except Exception as e:
        print(f"❌ Error en restauración: {e}")
        return False

def eliminar_empleado_fisico(empleado_id: int, confirmacion: bool = False) -> bool:
    """Eliminación física - CUIDADO: No se puede deshacer"""
    
    if not confirmacion:
        print("⚠️ ADVERTENCIA: Eliminación física requiere confirmación explícita")
        print("   Esta operación NO SE PUEDE DESHACER")
        print("   Use confirmacion=True para proceder")
        return False
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Obtener datos del empleado antes de eliminar
            cursor.execute("SELECT * FROM empleados WHERE id = ?", (empleado_id,))
            empleado = cursor.fetchone()
            
            if not empleado:
                print(f"❌ Empleado ID {empleado_id} no encontrado")
                return False
            
            datos_empleado = dict(empleado)
            
            # Registrar auditoría ANTES de eliminar
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_anteriores)
                VALUES (?, 'HARD_DELETE', ?)
            """, (empleado_id, json.dumps(datos_empleado)))
            
            # Eliminar físicamente
            cursor.execute("DELETE FROM empleados WHERE id = ?", (empleado_id,))
            
            conn.commit()
            print(f"🚨 Empleado ID {empleado_id} ELIMINADO FÍSICAMENTE")
            print(f"👤 Empleado eliminado: {datos_empleado['nombre']} {datos_empleado['apellido']}")
            print("⚠️ Esta operación no se puede deshacer")
            return True
            
    except Exception as e:
        print(f"❌ Error en eliminación física: {e}")
        return False

def obtener_empleados_inactivos() -> List[Dict[str, Any]]:
    """Obtiene lista de empleados inactivos para gestión"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT e.*, a.timestamp as fecha_eliminacion, a.datos_nuevos as motivo_json
            FROM empleados e
            LEFT JOIN auditoria_empleados a ON e.id = a.empleado_id 
                AND a.operacion = 'SOFT_DELETE'
                AND a.timestamp = (
                    SELECT MAX(timestamp) 
                    FROM auditoria_empleados a2 
                    WHERE a2.empleado_id = e.id AND a2.operacion = 'SOFT_DELETE'
                )
            WHERE e.activo = 0
            ORDER BY a.timestamp DESC
        """)
        
        return [dict(row) for row in cursor.fetchall()]

# 🧪 PRUEBAS DE ELIMINACIÓN
print("🧪 PROBANDO OPERACIONES DELETE")
print("=" * 50)

# Crear empleado temporal para pruebas
empleado_temporal = {
    'nombre': 'Pedro',
    'apellido': 'Temporal',
    'email': 'pedro.temporal@test.com',
    'salario': 50000.00,
    'departamento': 'Pruebas',
    'fecha_ingreso': '2024-01-01',
    'años_experiencia': 2
}

id_temporal = crear_empleado(empleado_temporal)

if id_temporal:
    print(f"\\n📝 EMPLEADO TEMPORAL CREADO (ID: {id_temporal})")
    
    # Mostrar empleados activos antes de eliminación
    print(f"\\n👥 EMPLEADOS ACTIVOS ANTES:")
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
        activos_antes = cursor.fetchone()[0]
        print(f"   Total activos: {activos_antes}")
    
    # Eliminación lógica
    print(f"\\n🗑️ ELIMINACIÓN LÓGICA:")
    eliminar_empleado_logico(id_temporal, "Prueba de eliminación lógica")
    
    # Mostrar empleados activos después
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
        activos_despues = cursor.fetchone()[0]
        print(f"   Total activos después: {activos_despues}")
    
    # Mostrar empleados inactivos
    print(f"\\n👻 EMPLEADOS INACTIVOS:")
    inactivos = obtener_empleados_inactivos()
    for emp in inactivos:
        motivo_data = json.loads(emp['motivo_json']) if emp['motivo_json'] else {}
        motivo = motivo_data.get('motivo', 'No especificado')
        print(f"   • {emp['nombre']} {emp['apellido']} - Motivo: {motivo}")
    
    # Restauración
    print(f"\\n🔄 RESTAURANDO EMPLEADO:")
    restaurar_empleado(id_temporal)
    
    # Verificar restauración
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
        activos_final = cursor.fetchone()[0]
        print(f"   Total activos después de restauración: {activos_final}")
    
    # Eliminación física (para limpieza)
    print(f"\\n🚨 ELIMINACIÓN FÍSICA (PRUEBA):")
    print("   Nota: En producción, esta operación sería muy restringida")
    eliminar_empleado_fisico(id_temporal, confirmacion=True)

print(f"\\n📊 RESUMEN FINAL DE EMPLEADOS:")
stats_finales = obtener_estadisticas_empleados()
print(f"   👥 Total empleados activos: {stats_finales['generales']['total_empleados']}")

print("\\n🧠 CONCEPTOS CLAVE DELETE:")
print("✅ Soft Delete: Preserva datos con flag inactivo")
print("✅ Hard Delete: Eliminación física irreversible")
print("✅ Auditoría completa de eliminaciones")
print("✅ Función de restauración para soft deletes")
print("✅ Gestión de empleados inactivos")

## 🏗️ 3. PATRONES DE DISEÑO PROFESIONALES

### 🎯 DAO (Data Access Object) Pattern

El patrón DAO encapsula todas las operaciones de base de datos en una clase organizada, facilitando el mantenimiento y reutilización del código.

In [ ]:
# 🏗️ CLASE DAO - DATA ACCESS OBJECT
class EmpleadoDAO:
    """Data Access Object para empleados - Patrón profesional"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self._inicializar_tabla()
    
    def _inicializar_tabla(self):
        """Inicializa las tablas necesarias"""
        # Ya las tenemos creadas, pero podríamos verificar/crear aquí
        pass
    
    # === OPERACIONES CRUD ===
    
    def crear(self, empleado: Dict[str, Any]) -> Optional[int]:
        """Crea un nuevo empleado"""
        return crear_empleado(empleado)
    
    def obtener_por_id(self, empleado_id: int) -> Optional[Dict[str, Any]]:
        """Obtiene empleado por ID"""
        return obtener_empleado_por_id(empleado_id)
    
    def buscar(self, filtros: Dict[str, Any] = None) -> List[Dict[str, Any]]:
        """Busca empleados con filtros"""
        return buscar_empleados(filtros)
    
    def actualizar(self, empleado_id: int, datos: Dict[str, Any]) -> bool:
        """Actualiza un empleado"""
        return actualizar_empleado(empleado_id, datos)
    
    def eliminar_logico(self, empleado_id: int, motivo: str = "") -> bool:
        """Eliminación lógica"""
        return eliminar_empleado_logico(empleado_id, motivo)
    
    def restaurar(self, empleado_id: int) -> bool:
        """Restaura empleado inactivo"""
        return restaurar_empleado(empleado_id)
    
    # === MÉTODOS DE NEGOCIO ESPECÍFICOS ===
    
    def obtener_por_departamento(self, departamento: str) -> List[Dict[str, Any]]:
        """Obtiene empleados de un departamento específico"""
        return self.buscar({'departamento': departamento})
    
    def obtener_por_rango_salarial(self, min_salario: float, max_salario: float) -> List[Dict[str, Any]]:
        """Obtiene empleados en rango salarial"""
        return self.buscar({'salario': {'min': min_salario, 'max': max_salario}})
    
    def contar_por_departamento(self) -> Dict[str, int]:
        """Cuenta empleados por departamento"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT departamento, COUNT(*) as total
                FROM empleados 
                WHERE activo = 1
                GROUP BY departamento
                ORDER BY total DESC
            """)
            
            return {row['departamento']: row['total'] for row in cursor.fetchall()}
    
    def obtener_top_salarios(self, limite: int = 5) -> List[Dict[str, Any]]:
        """Obtiene empleados con mejores salarios"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT * FROM empleados 
                WHERE activo = 1
                ORDER BY salario DESC 
                LIMIT ?
            """, (limite,))
            
            return [dict(row) for row in cursor.fetchall()]
    
    def obtener_antiguedad_promedio_departamento(self) -> Dict[str, float]:
        """Calcula antigüedad promedio por departamento"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT 
                    departamento,
                    ROUND(AVG(julianday('now') - julianday(fecha_ingreso)) / 365.25, 1) as años_promedio
                FROM empleados 
                WHERE activo = 1
                GROUP BY departamento
                ORDER BY años_promedio DESC
            """)
            
            return {row['departamento']: row['años_promedio'] for row in cursor.fetchall()}
    
    def generar_reporte_completo(self) -> Dict[str, Any]:
        """Genera reporte completo del sistema"""
        return {
            'estadisticas_generales': obtener_estadisticas_empleados(),
            'conteo_por_departamento': self.contar_por_departamento(),
            'top_salarios': self.obtener_top_salarios(3),
            'antiguedad_departamentos': self.obtener_antiguedad_promedio_departamento(),
            'timestamp': datetime.now().isoformat()
        }

class SensorDAO:
    """DAO para sensores industriales"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self._inicializar_tablas()
    
    def _inicializar_tablas(self):
        """Crea tablas de sensores si no existen"""
        with obtener_conexion_segura(self.db_path) as conn:
            conn.executescript("""
                CREATE TABLE IF NOT EXISTS sensores (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    nombre TEXT NOT NULL,
                    tipo TEXT NOT NULL,
                    ubicacion TEXT,
                    unidad_medida TEXT,
                    valor_min REAL,
                    valor_max REAL,
                    activo BOOLEAN DEFAULT 1,
                    fecha_creacion DATETIME DEFAULT CURRENT_TIMESTAMP
                );
                
                CREATE TABLE IF NOT EXISTS lecturas_sensores (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    sensor_id INTEGER,
                    valor REAL NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    estado TEXT DEFAULT 'NORMAL',
                    FOREIGN KEY (sensor_id) REFERENCES sensores(id)
                );
                
                CREATE INDEX IF NOT EXISTS idx_lecturas_sensor_time 
                ON lecturas_sensores(sensor_id, timestamp);
            """)
            conn.commit()
    
    def crear_sensor(self, sensor_data: Dict[str, Any]) -> int:
        """Crea un nuevo sensor"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                INSERT INTO sensores (nombre, tipo, ubicacion, unidad_medida, valor_min, valor_max)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                sensor_data['nombre'], sensor_data['tipo'],
                sensor_data['ubicacion'], sensor_data['unidad_medida'],
                sensor_data['valor_min'], sensor_data['valor_max']
            ))
            conn.commit()
            return cursor.lastrowid
    
    def registrar_lectura(self, sensor_id: int, valor: float) -> bool:
        """Registra una lectura del sensor"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            
            # Obtener límites del sensor
            cursor.execute("SELECT valor_min, valor_max FROM sensores WHERE id = ?", (sensor_id,))
            sensor = cursor.fetchone()
            
            if not sensor:
                return False
            
            valor_min, valor_max = sensor
            
            # Determinar estado
            if valor < valor_min:
                estado = 'BAJO'
            elif valor > valor_max:
                estado = 'ALTO'
            else:
                estado = 'NORMAL'
            
            # Registrar lectura
            cursor.execute("""
                INSERT INTO lecturas_sensores (sensor_id, valor, estado)
                VALUES (?, ?, ?)
            """, (sensor_id, valor, estado))
            
            conn.commit()
            return True
    
    def obtener_lecturas_recientes(self, sensor_id: int, limite: int = 10) -> List[Dict[str, Any]]:
        """Obtiene las lecturas más recientes de un sensor"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT * FROM lecturas_sensores 
                WHERE sensor_id = ?
                ORDER BY timestamp DESC 
                LIMIT ?
            """, (sensor_id, limite))
            
            return [dict(row) for row in cursor.fetchall()]

# 🧪 DEMO DEL PATRÓN DAO
print("🧪 DEMOSTRANDO PATRÓN DAO")
print("=" * 50)

# Crear instancia del DAO
empleado_dao = EmpleadoDAO('empresa_industrial.db')
sensor_dao = SensorDAO('empresa_industrial.db')

print("✅ DAOs inicializados")

# Usar métodos de negocio específicos
print(f"\\n📊 CONTEO POR DEPARTAMENTO:")
conteo_depto = empleado_dao.contar_por_departamento()
for depto, cantidad in conteo_depto.items():
    print(f"   🏢 {depto}: {cantidad} empleados")

print(f"\\n🏆 TOP 3 SALARIOS:")
top_salarios = empleado_dao.obtener_top_salarios(3)
for i, emp in enumerate(top_salarios, 1):
    print(f"   #{i} {emp['nombre']} {emp['apellido']}: ${emp['salario']:,.2f}")

print(f"\\n⏰ ANTIGÜEDAD PROMEDIO POR DEPARTAMENTO:")
antiguedad = empleado_dao.obtener_antiguedad_promedio_departamento()
for depto, años in antiguedad.items():
    print(f"   🏢 {depto}: {años} años promedio")

# Demo de sensores
print(f"\\n🔧 CREANDO SENSOR DE TEMPERATURA:")
sensor_temp = {
    'nombre': 'Temperatura Reactor Principal',
    'tipo': 'Temperatura',
    'ubicacion': 'Planta - Reactor 1',
    'unidad_medida': '°C',
    'valor_min': 20.0,
    'valor_max': 80.0
}

sensor_id = sensor_dao.crear_sensor(sensor_temp)
print(f"   ✅ Sensor creado con ID: {sensor_id}")

# Simular algunas lecturas
print(f"\\n📊 SIMULANDO LECTURAS:")
lecturas_demo = [75.5, 82.1, 78.3, 85.0, 76.8]  # Algunos valores fuera de rango
for valor in lecturas_demo:
    sensor_dao.registrar_lectura(sensor_id, valor)
    print(f"   📈 Lectura registrada: {valor}°C")

# Obtener lecturas recientes
print(f"\\n📋 LECTURAS RECIENTES:")
lecturas = sensor_dao.obtener_lecturas_recientes(sensor_id, 5)
for lectura in lecturas:
    timestamp = lectura['timestamp']
    valor = lectura['valor']
    estado = lectura['estado']
    icono = "🔴" if estado == 'ALTO' else "🔵" if estado == 'BAJO' else "🟢"
    print(f"   {icono} {timestamp}: {valor}°C ({estado})")

print("\\n🧠 VENTAJAS DEL PATRÓN DAO:")
print("✅ Encapsulación de operaciones de BD")
print("✅ Métodos de negocio específicos")
print("✅ Reutilización de código")
print("✅ Fácil testing y mantenimiento")
print("✅ Separación de responsabilidades")

## 📊 4. PANDAS + SQL - ANÁLISIS HÍBRIDO

### 🔗 La Potencia de Combinar SQL y Pandas

Pandas ofrece funciones específicas para trabajar con SQL, permitiendo aprovechar lo mejor de ambos mundos: la eficiencia de SQL y las capacidades analíticas de Pandas.

In [ ]:
# 📊 ANÁLISIS HÍBRIDO PANDAS + SQL
class AnalisisIndustrial:
    """Clase para análisis avanzado de datos industriales"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
    
    def obtener_conexion_pandas(self):
        """Conexión optimizada para pandas"""
        return sqlite3.connect(self.db_path)
    
    def empleados_to_dataframe(self) -> pd.DataFrame:
        """Convierte empleados a DataFrame para análisis"""
        query = """
            SELECT 
                nombre || ' ' || apellido as nombre_completo,
                email, salario, departamento, 
                fecha_ingreso, años_experiencia,
                julianday('now') - julianday(fecha_ingreso) as dias_empresa
            FROM empleados 
            WHERE activo = 1
        """
        
        with self.obtener_conexion_pandas() as conn:
            df = pd.read_sql_query(query, conn)
            
            # Conversiones de tipos
            df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso'])
            df['años_empresa'] = df['dias_empresa'] / 365.25
            
            return df
    
    def analisis_salarial_avanzado(self) -> Dict[str, Any]:
        """Análisis salarial usando pandas"""
        df = self.empleados_to_dataframe()
        
        # Estadísticas descriptivas completas
        stats_salario = df['salario'].describe()
        
        # Análisis por departamento
        depto_analysis = df.groupby('departamento').agg({
            'salario': ['mean', 'median', 'std', 'min', 'max', 'count'],
            'años_experiencia': 'mean',
            'años_empresa': 'mean'
        }).round(2)
        
        # Correlaciones
        correlaciones = df[['salario', 'años_experiencia', 'años_empresa']].corr()
        
        # Detección de outliers (usando IQR)
        Q1 = df['salario'].quantile(0.25)
        Q3 = df['salario'].quantile(0.75)
        IQR = Q3 - Q1
        outliers_inferior = Q1 - 1.5 * IQR
        outliers_superior = Q3 + 1.5 * IQR
        
        outliers = df[(df['salario'] < outliers_inferior) | (df['salario'] > outliers_superior)]
        
        return {
            'estadisticas_descriptivas': stats_salario.to_dict(),
            'analisis_departamental': depto_analysis.to_dict(),
            'correlaciones': correlaciones.to_dict(),
            'outliers_salariales': outliers[['nombre_completo', 'departamento', 'salario']].to_dict('records'),
            'resumen': {
                'total_empleados': len(df),
                'salario_promedio': df['salario'].mean(),
                'brecha_salarial': df['salario'].max() - df['salario'].min(),
                'coef_variacion': (df['salario'].std() / df['salario'].mean()) * 100
            }
        }
    
    def tendencias_temporales(self) -> pd.DataFrame:
        """Análisis de tendencias temporales de contratación"""
        df = self.empleados_to_dataframe()
        
        # Crear series temporal de contrataciones
        df['año_ingreso'] = df['fecha_ingreso'].dt.year
        df['mes_ingreso'] = df['fecha_ingreso'].dt.month
        
        # Análisis mensual
        contrataciones_mes = df.groupby(['año_ingreso', 'mes_ingreso']).agg({
            'nombre_completo': 'count',
            'salario': 'mean'
        }).rename(columns={
            'nombre_completo': 'contrataciones',
            'salario': 'salario_promedio_ingreso'
        })
        
        # Análisis anual
        contrataciones_año = df.groupby('año_ingreso').agg({
            'nombre_completo': 'count',
            'salario': ['mean', 'median'],
            'años_experiencia': 'mean'
        })
        
        return {
            'mensual': contrataciones_mes.reset_index(),
            'anual': contrataciones_año.reset_index()
        }
    
    def generar_reporte_excel_avanzado(self, archivo: str):
        """Genera reporte Excel con múltiples hojas y análisis"""
        
        # Obtener datos
        df_empleados = self.empleados_to_dataframe()
        analisis_salarial = self.analisis_salarial_avanzado()
        tendencias = self.tendencias_temporales()
        
        # Crear archivo Excel con múltiples hojas
        with pd.ExcelWriter(archivo, engine='openpyxl') as writer:
            
            # Hoja 1: Datos crudos
            df_empleados.to_excel(writer, sheet_name='Datos Empleados', index=False)
            
            # Hoja 2: Análisis departamental
            depto_df = pd.DataFrame(analisis_salarial['analisis_departamental']).T
            depto_df.to_excel(writer, sheet_name='Análisis Departamental')
            
            # Hoja 3: Tendencias anuales
            tendencias['anual'].to_excel(writer, sheet_name='Tendencias Anuales', index=False)
            
            # Hoja 4: Outliers
            outliers_df = pd.DataFrame(analisis_salarial['outliers_salariales'])
            if not outliers_df.empty:
                outliers_df.to_excel(writer, sheet_name='Outliers Salariales', index=False)
            
            # Hoja 5: Resumen ejecutivo
            resumen_df = pd.DataFrame([analisis_salarial['resumen']])
            resumen_df.to_excel(writer, sheet_name='Resumen Ejecutivo', index=False)
        
        print(f"✅ Reporte Excel generado: {archivo}")
        return archivo
    
    def dataframe_to_sql_demo(self):
        """Demuestra cómo insertar DataFrame en SQL"""
        
        # Crear DataFrame de ejemplo (nuevos empleados)
        nuevos_empleados = pd.DataFrame([
            {
                'nombre': 'Sofia', 'apellido': 'Ramirez', 
                'email': 'sofia.ramirez@industrial.com',
                'salario': 71000, 'departamento': 'Automatización',
                'fecha_ingreso': '2024-06-01', 'años_experiencia': 4
            },
            {
                'nombre': 'Miguel', 'apellido': 'Torres',
                'email': 'miguel.torres@industrial.com', 
                'salario': 63000, 'departamento': 'Calidad',
                'fecha_ingreso': '2024-06-15', 'años_experiencia': 6
            }
        ])
        
        print("📊 INSERTANDO DATAFRAME EN SQL:")
        print(nuevos_empleados)
        
        # Insertar en base de datos
        with self.obtener_conexion_pandas() as conn:
            # if_exists='append' agrega a tabla existente
            nuevos_empleados.to_sql('empleados', conn, if_exists='append', index=False)
        
        print("✅ DataFrame insertado en tabla 'empleados'")
        return len(nuevos_empleados)

# 🧪 DEMO DE ANÁLISIS HÍBRIDO
print("🧪 DEMOSTRANDO ANÁLISIS HÍBRIDO PANDAS + SQL")
print("=" * 60)

# Crear instancia del analizador
analizador = AnalisisIndustrial('empresa_industrial.db')

# 1. Convertir a DataFrame
print("\\n📊 CONVIRTIENDO EMPLEADOS A DATAFRAME:")
df_empleados = analizador.empleados_to_dataframe()
print(f"   📈 Shape: {df_empleados.shape}")
print(f"   📋 Columnas: {list(df_empleados.columns)}")

# Mostrar primeras filas
print("\\n👀 PRIMERAS 3 FILAS:")
print(df_empleados.head(3).to_string())

# 2. Análisis salarial avanzado
print("\\n💰 ANÁLISIS SALARIAL AVANZADO:")
analisis = analizador.analisis_salarial_avanzado()

print(f"\\n📊 ESTADÍSTICAS DESCRIPTIVAS:")
stats = analisis['estadisticas_descriptivas']
print(f"   💰 Promedio: ${stats['mean']:,.2f}")
print(f"   📈 Mediana: ${stats['50%']:,.2f}")
print(f"   📊 Desviación estándar: ${stats['std']:,.2f}")
print(f"   📉 Mínimo: ${stats['min']:,.2f}")
print(f"   📈 Máximo: ${stats['max']:,.2f}")

print(f"\\n🎯 RESUMEN EJECUTIVO:")
resumen = analisis['resumen']
print(f"   👥 Total empleados: {resumen['total_empleados']}")
print(f"   💰 Salario promedio: ${resumen['salario_promedio']:,.2f}")
print(f"   📊 Brecha salarial: ${resumen['brecha_salarial']:,.2f}")
print(f"   📈 Coeficiente variación: {resumen['coef_variacion']:.1f}%")

# Mostrar outliers si existen
outliers = analisis['outliers_salariales']
if outliers:
    print(f"\\n🔍 OUTLIERS SALARIALES:")
    for outlier in outliers:
        print(f"   • {outlier['nombre_completo']} ({outlier['departamento']}): ${outlier['salario']:,.2f}")
else:
    print("\\n✅ No se detectaron outliers salariales")

# 3. Correlaciones
print(f"\\n🔗 CORRELACIONES:")
correlaciones = analisis['correlaciones']
print(f"   📊 Salario vs Experiencia: {correlaciones['salario']['años_experiencia']:.3f}")
print(f"   📊 Salario vs Años Empresa: {correlaciones['salario']['años_empresa']:.3f}")
print(f"   📊 Experiencia vs Años Empresa: {correlaciones['años_experiencia']['años_empresa']:.3f}")

# 4. Demo de DataFrame to SQL
print(f"\\n💾 DEMO: DATAFRAME TO SQL")
empleados_agregados = analizador.dataframe_to_sql_demo()
print(f"   ✅ {empleados_agregados} empleados agregados desde DataFrame")

# 5. Generar reporte Excel
print(f"\\n📄 GENERANDO REPORTE EXCEL AVANZADO:")
archivo_excel = "reporte_empleados_avanzado.xlsx"
analizador.generar_reporte_excel_avanzado(archivo_excel)

# Verificar total actualizado
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
    total_final = cursor.fetchone()[0]
    print(f"\\n📈 Total empleados después de inserciones: {total_final}")

print("\\n🧠 VENTAJAS DEL ANÁLISIS HÍBRIDO:")
print("✅ SQL para consultas eficientes")
print("✅ Pandas para análisis estadístico avanzado")
print("✅ Exportación directa a Excel/CSV")
print("✅ Detección automática de outliers")
print("✅ Cálculo de correlaciones")
print("✅ Inserción masiva desde DataFrames")

## 🎓 CONSOLIDACIÓN Y PRÓXIMOS PASOS

### ✅ CONCEPTOS DOMINADOS EN ESTE MÓDULO

**🔌 Integración Python-SQLite:**
- Context managers para conexiones seguras
- Manejo robusto de errores y transacciones
- Configuración optimizada de conexiones

**📊 Operaciones CRUD Avanzadas:**
- Inserción con validación y auditoría
- Consultas dinámicas con filtros flexibles
- Actualización masiva con transacciones
- Eliminación lógica vs física

**🏗️ Patrones de Diseño:**
- DAO (Data Access Object) pattern
- Encapsulación de lógica de negocio
- Separación de responsabilidades
- Métodos especializados por dominio

**📈 Análisis Híbrido Pandas-SQL:**
- Conversión SQL → DataFrame
- Estadísticas descriptivas avanzadas
- Detección de outliers automática
- Exportación a múltiples formatos
- Inserción masiva desde DataFrames

### 🎯 APLICACIONES INDUSTRIALES DESARROLLADAS

1. **Sistema de Gestión de Empleados**
   - CRUD completo con auditoría
   - Búsquedas avanzadas y filtros
   - Análisis estadístico de recursos humanos

2. **Sistema de Monitoreo de Sensores**
   - Registro automático de lecturas
   - Detección de valores fuera de rango
   - Historial temporal de mediciones

3. **Generador de Reportes Automatizado**
   - Exportación a Excel multi-hoja
   - Análisis de tendencias temporales
   - Métricas de negocio automatizadas

---

### 🧠 EVALUACIÓN FINAL - VERIFIQUE SU DOMINIO

**Marque ✅ cada concepto que domina completamente:**

**🔌 CONEXIONES Y MANEJO:**
- [ ] Uso de context managers para SQLite
- [ ] Manejo adecuado de errores de BD
- [ ] Configuración de row_factory
- [ ] Transacciones seguras

**📊 OPERACIONES CRUD:**
- [ ] Inserción con validación de integridad
- [ ] Consultas dinámicas con filtros
- [ ] Actualización con auditoría
- [ ] Eliminación lógica vs física

**🏗️ PATRONES PROFESIONALES:**
- [ ] Implementación del patrón DAO
- [ ] Encapsulación de lógica de negocio
- [ ] Métodos especializados por dominio
- [ ] Separación de responsabilidades

**📈 ANÁLISIS AVANZADO:**
- [ ] Integración Pandas + SQL
- [ ] Análisis estadístico de datos
- [ ] Exportación a múltiples formatos
- [ ] Detección automática de outliers

**💼 APLICACIONES INDUSTRIALES:**
- [ ] Sistema de gestión de empleados
- [ ] Monitoreo de sensores IoT
- [ ] Generación automática de reportes
- [ ] Dashboard de métricas empresariales

---

### 🚀 ROADMAP - PRÓXIMOS SUBMÓDULOS

**🎯 SUBMÓDULO 3.3: CONSULTAS AVANZADAS Y OPTIMIZACIÓN**
- Views y stored procedures
- Triggers para automatización
- Optimización de consultas complejas
- Análisis de performance con EXPLAIN

**🎯 SUBMÓDULO 3.4: ORM CON SQLALCHEMY**
- Models y relaciones complejas
- Migrations automáticas
- Query builder avanzado
- Integración con frameworks web

**🎯 SUBMÓDULO 3.5: APIS REST CON BASES DE DATOS**
- FastAPI + SQLAlchemy
- Endpoints CRUD automatizados
- Autenticación y autorización
- Deploy en producción

**🎯 PROYECTO FINAL FASE 3:**
- Sistema completo de monitoreo industrial
- Dashboard web en tiempo real
- APIs REST para integración
- Base de datos optimizada para producción

---

### ✅ CONFIRMACIÓN DE CONSOLIDACIÓN

**🔥 Para marcar este submódulo como CONSOLIDADO:**

1. ✅ Ejecute exitosamente todos los ejemplos del notebook
2. ✅ Implemente su propio DAO para un dominio diferente
3. ✅ Cree un análisis híbrido Pandas-SQL personalizado
4. ✅ Genere un reporte automatizado de sus propios datos
5. ✅ Marque todos los ítems de la evaluación

**📝 Escriba "CONSOLIDADO 3.2" cuando domine completamente:** _________

---

**🎉 ¡EXCELENTE PROGRESO!**

Has dominado la integración Python-SQLite, base fundamental para automatización industrial. Ahora estás listo para consultas avanzadas y optimización en el siguiente submódulo.

**💪 TU NIVEL ACTUAL:**
- ✅ Fundamentos SQL (Módulo 3.1)
- ✅ Integración Python-SQLite (Módulo 3.2)
- 🎯 **PRÓXIMO:** Consultas Avanzadas y Optimización (Módulo 3.3)